(cont:mm:paging)=
# Paging

## Paging - Avoiding Fragmentation

The fragmentation in
[\[fig:vm:fig2\]](#fig:vm:fig2){reference-type="autoref"
reference="fig:vm:fig2"} is termed *external fragmentation*, because the
memory wasted is *external* to the regions allocated. This situation can
be avoided by *compacting* memory---moving existing allocations around,
thereby consolidating multiple blocks of free memory into a single large
chunk. This is a slow process, requiring processes to be paused, large
amounts of memory to be copied, and base+bounds registers modified to
point to new locations[^2].

![image](../images/pb-figures/mm/virt-mem-map.png){height="8\\baselineskip"}

Instead, modern CPUs use *paged address translation*, which divides the
physical and virtual memory spaces into fixed-sized pages, typically
4KB, and provides a flexible mapping between virtual and physical pages,
as shown in [\[fig:vm:fig3\]](#fig:vm:fig3){reference-type="autoref"
reference="fig:vm:fig3"}. The operating system can then maintain a list
of free physical pages, and allocate them as needed. Because any
combination of physical pages may be used for an allocation request,
there is no external fragmentation, and a request will not fail as long
as there are enough free physical pages to fulfill it.

Instead, modern CPUs use *paged address translation*, which divides the
physical and virtual memory spaces into fixed-sized pages, typically
4KB, and provides a flexible mapping between virtual and physical pages,
as shown in [\[fig:vm:fig3\]](#fig:vm:fig3){reference-type="autoref"
reference="fig:vm:fig3"}. The operating system can then maintain a list
of free physical pages, and allocate them as needed. Because any
combination of physical pages may be used for an allocation request,
there is no external fragmentation, and a request will not fail as long
as there are enough free physical pages to fulfill it.